In [ ]:
import itertools
import pandas as pd
from datetime import datetime
import requests
import utils
import setups.emas as emas
import setups.stopgain as StopGain
import setups.stoploss as StopLoss

# Função para buscar dados históricos de velas
def fetch_candles(symbol, interval, start_str, end_str=None):
    url = 'https://api.binance.com/api/v3/klines'
    data = []
    limit = 1000
    start_time = int(pd.to_datetime(start_str).timestamp() * 1000)
    end_time = int(pd.to_datetime(end_str).timestamp() * 1000) if end_str else None

    while True:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'limit': limit
        }
        if end_time:
            params['endTime'] = end_time

        response = requests.get(url, params=params)
        new_data = response.json()
        if not new_data:
            break
        data.extend(new_data)
        start_time = new_data[-1][0] + 1

    columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
               'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
               'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(data, columns=columns)
    df = df[['open', 'high', 'low', 'close', 'open_time', 'close_time']]
    return df

# Definir os parâmetros e seus intervalos
param_grid = {
    'short_period': [9, 12, 14],
    'long_period': [21, 26, 28],
    'stoploss_candles': [10, 14, 20],
    'stopgain_percentage': [10, 18, 25]
}

# Gerar todas as combinações possíveis de parâmetros
param_combinations = list(itertools.product(*param_grid.values()))

best_performance = None
best_params = None

# Dados históricos de exemplo
start_date = '2022-12-20'
end_date = datetime.now().strftime('%Y-%m-%d')
ativo = 'BTCUSDT'
timeframe = '15m'
data = fetch_candles(ativo, timeframe, start_date, end_date)
data['close'] = data['close'].astype(float)
data['low'] = data['low'].astype(float)
data['high'] = data['high'].astype(float)

# Função para avaliar o desempenho da estratégia
def evaluate_performance(data, short_period, long_period, stoploss_candles, stopgain_percentage):
    saldo = 1000
    taxa_por_operacao = 0.0153
    comprado = False

    data['EMA_short'] = data['close'].ewm(span=short_period, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_period, adjust=False).mean()

    for i in range(max(short_period, long_period), len(data)):
        if comprado:
            if StopLoss.sell_stoploss(data['low'].iloc[i - 1], stoploss):
                loss_percentage = utils.calculate_loss_percentage(buy_price, stoploss)
                saldo -= saldo * ((loss_percentage + taxa_por_operacao) / 100)
                comprado = False
                continue

            elif StopGain.sell_stopgain(data['high'].iloc[i - 1], stopgain):
                profit = utils.calculate_gain_percentage(buy_price, stopgain)
                saldo += saldo * ((profit - taxa_por_operacao) / 100)
                comprado = False
                continue

        if not comprado:
            if data['EMA_long'].iloc[i - 2] < data['close'].iloc[i - 2]:
                if emas.buy_ema_breakout(data['EMA_short'].iloc[i - 2], data['EMA_short'].iloc[i - 3], data['high'].iloc[i - 2], data['high'].iloc[i - 1]):
                    buy_price = data['high'].iloc[i - 2]
                    stoploss = StopLoss.set_sell_stoploss_min_candles(data.iloc[i - stoploss_candles:i], stoploss_candles - 1)
                    if taxa_por_operacao != 0:
                        saldo -= saldo * taxa_por_operacao / 100
                    stopgain = StopGain.set_sell_stopgain_percentage(buy_price, stopgain_percentage)
                    comprado = True
                    continue

    performance_metric = saldo - 1000  # Exemplos de métrica de desempenho: saldo final menos saldo inicial
    return performance_metric

for combination in param_combinations:
    params = dict(zip(param_grid.keys(), combination))
    
    # Avaliar o desempenho da estratégia com os parâmetros atuais
    performance_score = evaluate_performance(data, params['short_period'], params['long_period'], params['stoploss_candles'], params['stopgain_percentage'])
    
    # Registrar a melhor combinação
    if best_performance is None or performance_score > best_performance:
        best_performance = performance_score
        best_params = params

print("Melhores parâmetros encontrados:")
print(best_params)
print("Melhor desempenho obtido:")
print(best_performance)
